In [2]:
from discord.ext import commands
import discord
import pickle
import csv
import os
import numpy as np
import scipy.stats as sps
import json


In [9]:
def elo_func(kost, kills, deaths, win):
    kost_mu = 0.55
    kost_sigma = 0.2
    dist = sps.norm(loc=kost_mu, scale=kost_sigma)
    p = dist.cdf(kost + ((kost - 0.55) ** 2) / 7 + (kost - 0.55) / 9)

    if win:
        if int(kost) == 1:
            kost_factor = 2
        else:    
            kost_factor = (np.sqrt(2) - 2 + 2 * (1 - np.sqrt(2)) * p) ** 2
        
        return round(150 * kost_factor + kills * 10 - deaths * 10)
    else:
        if int(kost) == 1:
            kost_factor = 0.4
        else:    
            kost_factor = -6 / 5 * p + 1.6
        
        return round(-100 * kost_factor + kills * 10 - deaths * 10)

print('KOST  --  K MULTIPLIER  --  Win Elo  --  Loss Elo')

kost = 0.00
for i in range(200):
    kost_mu = 0.55
    kost_sigma = 0.2
    dist = sps.norm(loc=kost_mu, scale=kost_sigma)
    p = dist.cdf(kost + ((kost - 0.55) ** 2) / 7 + (kost - 0.55) / 9)

    if int(kost) == 1:
        kost_factor = 2
    else:    
        kost_factor = (np.sqrt(2) - 2 + 2 * (1 - np.sqrt(2)) * p) ** 2
    print('{:.2f}         x {:.5f}                {}                   {}'.format(kost, kost_factor,
                                                                                   elo_func(kost, 0, 0, True),
                                                                                   elo_func(kost, 0, 0, False)))
    kost += 0.05
    if kost > 1.04:
        break

KOST  --  K MULTIPLIER  --  Win Elo  --  Loss Elo
0.00         x 0.34534                52                   -160
0.05         x 0.34770                52                   -159
0.10         x 0.35218                53                   -159
0.15         x 0.36036                54                   -158
0.20         x 0.37456                56                   -156
0.25         x 0.39812                60                   -153
0.30         x 0.43544                65                   -149
0.35         x 0.49190                74                   -143
0.40         x 0.57332                86                   -135
0.45         x 0.68466                103                   -125
0.50         x 0.82797                124                   -113
0.55         x 1.00000                150                   -100
0.60         x 1.19073                179                   -87
0.65         x 1.38440                208                   -74
0.70         x 1.56332                234          

In [1]:
class Player:
    def __init__(self, name: str, kost: float, kills: int, deaths: int, wins: int,
                 losses: int, points: int, elo: int):
        self.name = name
        self.kost = kost
        self.kills = kills
        self.deaths = deaths
        self.wins = wins
        self.losses = losses
        self.points = points
        self.elo = elo

with open('match.json') as match_json:
    match_data = json.load(match_json)

player_data = match_data['per_player']
fteam_win = bool(match_data['friendly_win'])

f_w = 0
e_w = 0

if fteam_win:
    f_w = 1
    e_w = 0
else:
    f_w = 0
    e_w = 1

player_list = []

for p_id in player_data:
    p_dic = player_data[p_id]

    if p_dic['team'] == 'friendly':
        try:
            player = Player(p_dic['name'], float(p_dic['kost'].replace('%', ''))/100, int(p_dic['kills']), int(p_dic['deaths']), f_w, e_w,
                            int(p_dic['score']), 0)
            player_list.append(player)
            print(player.kost)
        except:
            pass
    else:
        try:
            player = Player(p_dic['name'], float(p_dic['kost'].replace('%', ''))/100, int(p_dic['kills']), int(p_dic['deaths']), e_w, f_w,
                            int(p_dic['score']), 0)
            player_list.append(player)
            print(player.kost)
        except:
            pass
    

FileNotFoundError: [Errno 2] No such file or directory: 'match.json'